# Prepare the Readcount Data I got from David for the South Americans into a HDF5 compatible with HAPSBURG
Additonally: Also prepare the downsampled RC/PH HDF5 versions of high coverage SA Individuals
@ Author: Harald Ringbauer, 2019

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os  # For Saving to Folder
import pandas as pd
import h5py  # Python Package to do the HDF5.

import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
sys.path.append("./Python3/")  # Since now we are in the Root Directory
from hmm_inference import HMM_Analyze   # Do not move. Should be after sys.path..


print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


# Load the Data

In [17]:
def get_df_from_rc(path_ind = "../fromDavid/MA577_1240k_all.cnts"):
    df_rc = pd.read_csv(path_ind, header=None, sep=" ")
    df_rc.columns=["chr","pos", "ref", "alt", "A", "G", "C", "T"]
    print(f"Loaded {len(df_rc)} Markers")
    return df_rc

def creat_count_col(df_rc):
    """Add Fields for ref and alt Count"""
    df_rc["ref_count"]=0
    df_rc["alt_count"]=0

    for a in ["A", "G", "C", "T"]:
        idcs = df_rc["ref"]==a
        df_rc.loc[idcs,"ref_count"]=df_rc[a]

        idcs = df_rc["alt"]==a
        df_rc.loc[idcs,"alt_count"]=df_rc[a]
    mean_depth = np.mean(df_rc["alt_count"]+df_rc["ref_count"])
    print(f"Mean Depth: {mean_depth:.4f}")
    return df_rc

def save_hdf5(gt, ad, ref, alt, pos, ch, samples, path):
    """Create a new HDF5 File with Input Data.
    gt: Genotype data [l,k,2]
    ad: Allele depth [l,k,2]
    ref: Reference Allele [l]
    alt: Alternate Allele [l]
    pos: Position  [l]
    m: Map position [l]
    ch: Which chromosome [l]
    samples: Sample IDs [k]"""

    l, k, _ = np.shape(gt)  # Nr loci and Nr of Individuals

    if os.path.exists(path):  ### Do a Deletion of existing File there
        os.remove(path)

    dt = h5py.special_dtype(vlen=str)  # To have no problem with saving

    with h5py.File(path, 'w') as f0:
        ### Create all the Groups
        #f_map = f0.create_dataset("variants/MAP", (l,), dtype='f')
        f_ch = f0.create_dataset("variants/CHROM", (l,), dtype='i')
        f_ad = f0.create_dataset("calldata/AD", (l, k, 2), dtype='i')
        f_ref = f0.create_dataset("variants/REF", (l,), dtype=dt)
        f_alt = f0.create_dataset("variants/ALT", (l,), dtype=dt)
        f_pos = f0.create_dataset("variants/POS", (l,), dtype='i')
        f_gt = f0.create_dataset("calldata/GT", (l, k, 2), dtype='i')
        f_samples = f0.create_dataset("samples", (k,), dtype=dt)

        ### Save the Data
        #f_map[:] = rec
        f_ch[:] = ch
        f_ad[:] = ad
        f_ref[:] = ref.astype("S1")
        f_alt[:] = alt.astype("S1")
        f_pos[:] = pos
        f_gt[:] = gt
        f_samples[:] = np.array(samples).astype("S10")

    print(f"Successfully saved {k} individuals to: {path}")

### Do single Indivudal

In [18]:
def rc_to_hdf_1ind(path_ind, path_h5="./Data/SA_1240kHDF5/MA577_1240k.h5", iid="MA577_1240k"):
    """Produce HDF5 File from Readcount Data"""
    df_rc= get_df_from_rc(path_ind)
    df_rc = creat_count_col(df_rc)
    l = len(df_rc)
    k = 1

    ###
    gt = -np.ones((l,k,2), dtype="int8") # No genotypes
    ad = df_rc[["ref_count", "alt_count"]].values[:,None,:] # None for n=1 axis
    ref = df_rc["ref"].values
    alt = df_rc["alt"].values
    pos = df_rc["pos"].values
    ch = df_rc["chr"].values
    samples=[iid,]

    save_hdf5(gt, ad, ref, alt, pos, ch, samples, path_h5)
    print(f"Successfully saved to {path_h5}")

In [4]:
base_folder="../fromDavid/"
out_folder="./Data/SA_1240kHDF5/"

#iids=["IPY10", "IPK12", "MA577", "894", "895"]
iids=["Loschbour", "Stuttgart", "Ust_Ishim"]

paths = [os.path.join(base_folder, iid + "_1240k_all.cnts") for iid in iids]
paths_h5 = [os.path.join(out_folder,iid + ".h5") for iid in iids]

for i in range(len(paths)):
    rc_to_hdf_1ind(paths[i], path_h5=paths_h5[i], iid=iids[i])

Loaded 1115283 Markers
Mean Depth: 19.5486
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Loschbour.h5
Successfully saved to ./Data/SA_1240kHDF5/Loschbour.h5
Loaded 1115215 Markers
Mean Depth: 18.2127
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Stuttgart.h5
Successfully saved to ./Data/SA_1240kHDF5/Stuttgart.h5
Loaded 1115315 Markers
Mean Depth: 40.7515
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Ust_Ishim.h5
Successfully saved to ./Data/SA_1240kHDF5/Ust_Ishim.h5


### Do Individuals from Moreno-Mayer
Do the Individuals where David W. downloaded and processed the RC data. Only iids and the suffix are different

In [19]:
base_folder="../fromDavid/"
out_folder="./Data/SA_1240kHDF5/"

iids = ["A460", "Andaman", "5832", "Lovelock4", "Sumidouro4","Sumidouro5", 
        "Sumidouro6", "Sumidouro7", "19651", "AHUR_2064", "Lovelock1", "Lovelock2", "Lovelock3", "Aconcagua"]

paths = [os.path.join(base_folder, iid + "_final.bam.mpileup.cnts") for iid in iids]
paths_h5 = [os.path.join(out_folder, iid + ".h5") for iid in iids]

for i in range(len(paths)):
    rc_to_hdf_1ind(paths[i], path_h5=paths_h5[i], iid=iids[i])

Loaded 1115101 Markers
Mean Depth: 12.9466
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/A460.h5
Successfully saved to ./Data/SA_1240kHDF5/A460.h5
Loaded 1102888 Markers
Mean Depth: 22.4121
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Andaman.h5
Successfully saved to ./Data/SA_1240kHDF5/Andaman.h5
Loaded 912911 Markers
Mean Depth: 2.2065
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/5832.h5
Successfully saved to ./Data/SA_1240kHDF5/5832.h5
Loaded 543174 Markers
Mean Depth: 1.3750
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Lovelock4.h5
Successfully saved to ./Data/SA_1240kHDF5/Lovelock4.h5
Loaded 696732 Markers
Mean Depth: 1.6262
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Sumidouro4.h5
Successfully saved to ./Data/SA_1240kHDF5/Sumidouro4.h5
Loaded 1114912 Markers
Mean Depth: 18.8313
Successfully saved 1 individuals to: ./Data/SA_1240kHDF5/Sumidouro5.h5
Successfully saved to ./Data/SA_1240kHDF5/Sumidouro5.h5
Loaded 1008433 Mar

# Do multiple individuals
Did not work because different number of SNPs! Left here as a reminder!

# Prepare Downsampled HDF5
Idea: Have HDF5s with several individuals that are downsampled from one master Individual

In [11]:
def downsample_ad(ad, cov_frac):
    """Return downsampled Allele Depth
    ad: [l,2] Array of Allele Depths
    cov_frac: To which fraction to downsample"""
    assert(np.min(ad)>=0)   # Sanity Check
    ad = np.random.binomial(ad, cov_frac)
    return ad
    
def rc_to_hdf_cov_ind(base_folder, iid, path_hdf5, down_sampling_covs):
    """Create HDF5 from one Individual, downsampled to various levels.
    down_sampling_covs: Vector of coverages to downsamples to"""
    ### Produce the Sample Names
    samples = [f"{c:.4f}" for c in down_sampling_covs]
    
    ### Same as for 1 Individual, but as a Loop
    path = os.path.join(base_folder, iid + "_1240k_all.cnts")
    df_rc = get_df_from_rc(path)
    df_rc = creat_count_col(df_rc)

    ### Get Fields that are same for all Individuals (i.e. Array)
    ref = df_rc["ref"].values
    alt = df_rc["alt"].values
    pos = df_rc["pos"].values
    ch = df_rc["chr"].values

    ### Get Fields that are a matrix
    l = len(df_rc)
    k = len(down_sampling_covs)
    gt = -np.ones((l, k, 2), dtype="int8")            # No genotypes filled in!

    ad = df_rc[["ref_count", "alt_count"]].values     # Extract Allele Depth Field
    ads= [downsample_ad(ad, cov_frac) for cov_frac in down_sampling_covs]
    ad = np.stack(ads, axis=1)          # Combine the allele Depths (along axis 1 for individuals)
    
    assert(np.shape(ad)==np.shape(gt)) # Sanity Check
    save_hdf5(gt, ad, ref, alt, pos, ch, samples, path_hdf5)

In [12]:
### IPK12
#down_sampling_covs = np.geomspace(0.04, 1.0, 10)
#rc_to_hdf_cov_ind("../fromDavid/", "IPK12", path_hdf5="./Data/SA_1240kHDF5/IPK12_downsample.h5", 
#                  down_sampling_covs=down_sampling_covs)

down_sampling_covs=np.geomspace(0.01, 1.0, 10)
rc_to_hdf_cov_ind("../fromDavid/", "Stuttgart", path_hdf5="./Data/SA_1240kHDF5/Stuttgart_downsample.h5", 
                  down_sampling_covs=down_sampling_covs)

Loaded 1115215 Markers
Mean Depth: 18.2127
Successfully saved 10 individuals to: ./Data/SA_1240kHDF5/Stuttgart_downsample.h5


In [ ]:
down_sampling_covs=np.geomspace(0.01, 1.0, 10)
rc_to_hdf_cov_ind("../fromDavid/", "Loschbour", path_hdf5="./Data/SA_1240kHDF5/Loschbour_downsample.h5", 
                  down_sampling_covs=down_sampling_covs)

### Prepare Downsampled HDF5 with Pseudohaploid Genotypes

In [9]:
def downsample_ph(ad, cov_frac=1.0, mis_val=-1):
    """Downsample to pseudohaploid Genotypes
    Input: Allele Depths [l,2], cov_frac: Target Coverage (1.0x nothing set to 0)"""
    tot_cov = np.sum(ad, axis=1)
    cov0 = (tot_cov ==0)
    tot_cov[cov0]=1    # Fill up with fake value, will be replaced later!!
    p = ad[:,1] / tot_cov  # Fraction of derived Read per Locus
    
    gt = np.random.binomial(n=1, p=p) # The Pseudohaploid Genotypes
    gt = gt.astype("int8")
    
    ### Do the downsampling
    not_cov = np.random.random(size=len(gt)) > cov_frac  # Create missing mask
    gt[not_cov] = mis_val # Set missing
    gt[cov0] = mis_val # Set missing where no coverage
    
    ### Do the Duplication
    gt = np.tile(gt, (2,1)).T # Create (l,2) Array
    return gt

def rc_to_hdf_cov_ind_ph(base_folder, iid, path_hdf5, down_sampling_covs):
    """Create HDF5 from one Individual, downsampled to various levels.
    down_sampling_covs: Vector of coverages to downsamples to"""
    ### Produce the Sample Names
    samples = [f"{c:.4f}" for c in down_sampling_covs]
    
    ### Same as for 1 Individual, but as a Loop
    path = os.path.join(base_folder, iid + "_1240k_all.cnts")
    df_rc = get_df_from_rc(path)
    df_rc = creat_count_col(df_rc)

    ### Get Fields that are same for all Individuals (i.e. Array)
    ref = df_rc["ref"].values
    alt = df_rc["alt"].values
    pos = df_rc["pos"].values
    ch = df_rc["chr"].values

    ### Get Fields that are a matrix
    l = len(df_rc)
    k = len(down_sampling_covs)
    
    #gts = np.zeros((l, k, 2), dtype="int8")
    #gt = -np.ones((l, k, 2), dtype="int8")           # No genotypes filled in!

    ad = df_rc[["ref_count", "alt_count"]].values     # Extract Allele Depth Field
    
    gts= [downsample_ph(ad, cov_frac) for cov_frac in down_sampling_covs]
    gts = np.stack(gts, axis=1)            # Combine the allele Depths (along axis 1 for individuals)
    ads = np.tile(ad[:,None,:], (1, k, 1)) # Tile the Allele depths as well
    
    assert(np.shape(ads)==np.shape(gts)) # Sanity Check
    save_hdf5(gts, ads, ref, alt, pos, ch, samples, path_hdf5)

In [10]:
down_sampling_covs = np.linspace(0.3, 1.0, 8)
rc_to_hdf_cov_ind_ph("../fromDavid/", "Loschbour", path_hdf5="./Data/SA_1240kHDF5/Loschbour_downsample_ph.h5", 
                     down_sampling_covs=down_sampling_covs)

Loaded 1115283 Markers
Mean Depth: 19.5486
Successfully saved 8 individuals to: ./Data/SA_1240kHDF5/Loschbour_downsample_ph.h5


In [13]:
down_sampling_covs = np.linspace(0.3, 1.0, 8)
rc_to_hdf_cov_ind_ph("../fromDavid/", "Stuttgart", path_hdf5="./Data/SA_1240kHDF5/Stuttgart_downsample_ph.h5", 
                     down_sampling_covs=down_sampling_covs)

Loaded 1115215 Markers
Mean Depth: 18.2127
Successfully saved 8 individuals to: ./Data/SA_1240kHDF5/Stuttgart_downsample_ph.h5


# Area 51

In [53]:
down_sampling_covs = np.linspace(0.3, 1.0, 8)
down_sampling_covs

array([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [79]:
ad = np.array([[0,2],[0,2],[2,2]])
ad

array([[0, 2],
       [0, 2],
       [2, 2]])

In [83]:
np.shape(ad)

(3, 2)

In [85]:
b = np.tile(ad[:,None,:], (1,5,1))
np.shape(b)

(3, 5, 2)

In [87]:
b[:,2,:]

array([[0, 2],
       [0, 2],
       [2, 2]])

### Test whether Loading works properly

In [7]:
path_load = "./Data/SA_1240kHDF5/IPK12_downsample.h5"
#path_load = "./Data/SA_1240kHDF5/IPK12_downsample_ph.h5"
f = h5py.File(path_load, "r") # Load for Sanity Check. See below!
        
print("Loaded HDF5")
print("Loaded %i variants" % np.shape(f["calldata/GT"])[0])
print("Loaded %i individuals" % np.shape(f["calldata/GT"])[1])
print(list(f["calldata"].keys()))
print(list(f["variants"].keys()))

Loaded HDF5
Loaded 1114250 variants
Loaded 10 individuals
['AD', 'GT']
['ALT', 'CHROM', 'POS', 'REF']


In [4]:
f["samples"][:]

array(['IPK120.300', 'IPK120.400', 'IPK120.500', 'IPK120.600',
       'IPK120.700', 'IPK120.800', 'IPK120.900', 'IPK121.000'],
      dtype=object)

In [8]:
np.shape(f["calldata/GT"])

(1114250, 10, 2)

In [10]:
covs_gt = [np.mean(f["calldata/GT"][:,i,:]>-1) for i in range(len(f["samples"]))]
covs_gt

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [9]:
covs_ad = [np.mean(f["calldata/AD"][:,i,:]) * 2 for i in range(len(f["samples"]))]
covs_ad

[0.4171200358985865,
 0.5966363024455912,
 0.8540067309849675,
 1.223265873906215,
 1.747519856405654,
 2.4966748934260714,
 3.567838456360781,
 5.104206416872335,
 7.298326228404757,
 10.438989454790217]

In [13]:
10.438989454790217 * 0.04

0.4175595781916087

In [97]:
f["calldata/AD"][:10,1,:]

array([[ 0, 13],
       [ 6,  0],
       [12,  0],
       [15,  0],
       [ 9,  0],
       [10,  0],
       [11,  5],
       [ 3,  3],
       [ 3,  8],
       [ 0,  8]], dtype=int32)

In [105]:
f["calldata/GT"][:10,5,:]

array([[ 1,  1],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [-1, -1],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 1,  1],
       [-1, -1]], dtype=int32)

In [96]:
pd.value_counts(f["variants"]["CHROM"][:])

2     95335
1     90123
3     78552
6     76651
5     70233
4     69614
8     61970
7     60569
10    59543
11    55511
12    54531
9     51302
13    39491
14    36750
16    34954
15    34862
18    34309
17    29632
20    29427
19    18681
21    16234
22    15976
dtype: int64

In [34]:
np.where(f["samples"][:]=="MA577_1240")[0]

array([], dtype=int64)

In [74]:
iid="MA577_1240"

samples = f["samples"][:]
assert(len(samples) == np.shape(f["calldata/GT"])[1])  # Sanity Check

id_obs = np.where(samples == iid)[0]
if len(id_obs) == 0:
    raise RuntimeError(f"Individual {iid} not found in Samples Field")

In [76]:
id_obs[0]

0

In [41]:
f["samples"][:]

array(['IPY10', 'IPK12', 'MA577', '894', '895'], dtype=object)

In [ ]:
'IPY10', 'IPK12', 'MA577', '894', '895'

In [80]:
df_rc= get_df_from_rc(path_ind = "../fromDavid/IPY10_1240k_all.cnts")
df_rc = creat_count_col(df_rc)

Loaded 1114799 Markers


In [92]:
df_rc[df_rc["chr"]==10]

,chr,pos,ref,alt,A,G,C,T,ref_count,alt_count
654705,10,113934,A,C,6,0,0,0,6,0
654706,10,126070,C,T,0,0,0,13,0,13
654707,10,135656,T,G,0,0,0,12,12,0
654708,10,135853,A,G,13,0,0,0,13,0
654709,10,148325,A,G,9,0,0,0,9,0
654710,10,151997,T,C,0,0,0,18,18,0
654711,10,158202,C,T,0,0,7,0,7,0
654712,10,158946,C,T,0,0,19,0,19,0
654713,10,159076,G,A,0,14,0,0,14,0
654714,10,159404,G,A,0,16,0,0,16,0


In [66]:
df_t = get_df_from_rc(path_ind='../fromDavid/IPK12_1240k_all.cnts')
df_t = creat_count_col(df_t)

Loaded 1114250 Markers
Mean Depth: 10.4390


In [68]:
sum_cov = (df_t["ref_count"] + df_t["alt_count"])

In [71]:
sum_cov.value_counts()

9     104714
10    104050
8      99647
11     98571
7      88828
12     88270
13     77411
6      73016
14     63080
5      54729
15     50644
16     39521
4      36999
17     29702
18     21965
3      21329
19     15274
20     10828
2      10143
21      7226
22      4714
1       3911
23      3159
24      1928
0       1747
25      1179
26       661
27       402
28       225
29       159
30        81
31        57
32        36
33        16
34         7
35         6
37         5
38         2
40         2
47         1
52         1
55         1
46         1
36         1
64         1
dtype: int64

In [72]:
np.random.binomial(0,0)

0